In [2]:
 pip install imutils==0.5.3

  Using cached https://files.pythonhosted.org/packages/b5/94/46dcae8c061e28be31bcaa55c560cb30ee9403c9a4bb2659768ec1b9eb7d/imutils-0.5.3.tar.gz
  Created wheel for imutils: filename=imutils-0.5.3-cp37-none-any.whl size=25853 sha256=f7ecedd5caf2e38055d7cd7a7a778d242d74e9d99b2d8163b1a3d372559c98c8
  Stored in directory: C:\Users\Aravind\AppData\Local\pip\Cache\wheels\16\84\1f\bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [1]:
#installing the neccessary libraries 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os


In [2]:
#intializing the global variabels
EPOCH=20
INIT_LR=1e-4
bs=32

In [3]:
#assign the path of the dataset to the varibels 
dirc=r"C:\Users\Aravind\samples"
CATGS = ["with_mask", "without_mask"]

In [4]:
data=[]
labels=[]
#intializing the two list variables to store the images and the corresponding labels in the image
for category in CATGS:
        path=os.path.join(dirc,category)#joining the each folders path with already existing path of the directory
        for img in os.listdir(path):#travesrsing through the images in the list
            img_path=os.path.join(path,img)
            image=load_img(img_path,target_size=(224,224))#converting the image to uniform size(224,224)
            image=img_to_array(image)#converting the image into array of pixels
            image=preprocess_input(image)#preprocessing the array of pixel

            data.append(image)#appending the processed array of the pixel in the list 
            labels.append(category)#appending the corresponding category

C:\Users\Aravind\Anaconda3\lib\site-packages\PIL\Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [5]:
#converting the categorical variable using the label binarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels=to_categorical(labels)# to categorical used to convert the encoded vaues in the form of the matrix

In [6]:
#converting our list to np arrays because neural network procces only the array
data=np.array(data,dtype="float32")
labels=np.array(labels)
#separataing the data set for the trainning and testing
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=1)

In [7]:
#image agumentaion is done because we have around 1000 records for trainnig and testing
aug=ImageDataGenerator(rotation_range=20,
                       zoom_range=0.15,
                       width_shift_range=0.2,
                       height_shift_range=0.2,
                       shear_range=0.15,
                       horizontal_flip=True,
                       fill_mode='nearest' )

In [8]:
#creating our base model using mobilenets
baseModel=MobileNetV2(weights="imagenet",include_top=False,#include_top used to represent the fully connected layer at the top. here the value is given false because we are doing that mannualy                     
input_tensor=Input(shape=(224,224,3)))#intialzing the input dimension of the image
headModel=baseModel.output#we are taking the output of the basemodel as the input
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
#(headModel)using this we are connnecting the output of the headmodel to this layer
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(128,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation="softmax")(headModel)


model=Model(inputs=baseModel.input,outputs=headModel)

C:\Users\Aravind\Anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
for layer in baseModel.layers:
    layer.trainable=False

In [11]:
print("[info] compiling model..")
opt=Adam(lr=INIT_LR,decay=INIT_LR/EPOCH)#intializing the adam optimizer for optimization of our model
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])#compiling our model

[info] compiling model..


print(len(trainx)//bs)
print(len(testx)//bs)


In [12]:
print("[info] traininf head...")
#fitting our model to our variables
h=model.fit(aug.flow(trainx,trainy,batch_size=bs),
            steps_per_epoch=len(trainx) //bs,
            validation_data=(testx,testy),
            validation_steps=len(testx)//bs,
            epochs=EPOCH
            )



[info] traininf head...
Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
95/95 [==============================] - 2601s 27s/step - loss: 0.5387 - acc: 0.7317 - val_loss: 0.3911 - val_acc: 0.8540
Epoch 2/20
95/95 [==============================] - 864s 9s/step - loss: 0.2825 - acc: 0.8869 - val_loss: 0.3625 - val_acc: 0.8540
Epoch 3/20
95/95 [==============================] - 879s 9s/step - loss: 0.2327 - acc: 0.9127 - val_loss: 0.3010 - val_acc: 0.8879
Epoch 4/20
95/95 [==============================] - 885s 9s/step - loss: 0.2029 - acc: 0.9295 - val_loss: 0.2402 - val_acc: 0.9126
Epoch 5/20
95/95 [==============================] - 27308s 287s/step - loss: 0.1478 - acc: 0.9509 - val_loss: 0.2815 - val_acc: 0.8996
Epoch 6/20
95/95 [==============================] - 1119s 12s/step - loss: 0.1503 - acc: 0.9479 - val_loss: 0.2635 - val_acc: 0.9074
Epoch 7/20
95/95 [==============================] - 854s 9s/step - loss: 0.1622 - acc: 0

In [16]:
predIdxs = model.predict(testx, batch_size=bs)
predIdxs = model.predict(testx, batch_size=bs)
#print(classification_report(testy.argmax(axis=1),predIdxs,target_names=lb.classes_))

print("[INFO]saving mask detector model.....")
#saving our model 
model.save("facemask_detector.model",save_format="h5")#among save model and h5 format h5 is lessweighted

[INFO]saving mask detector model.....
